In [1]:
import dill
from copy import deepcopy
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn

from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import BucketIterator
from torchtext.legacy.data import Iterator

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amole\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
RANDOM_SEED = 2021
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DATA_PATH = "data/processed/"

In [3]:
# 데이터 불러오기
TEXT = Field(
    sequential=True,
    use_vocab=True,
    tokenize=word_tokenize,
    lower=True,
    batch_first=True,
)
LABEL = Field(
    sequential=False,
    use_vocab=False,
    batch_first=True,
)


cola_train_data, cola_valid_data, cola_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="cola_train.tsv",
    validation="cola_valid.tsv",
    test="cola_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1
)

TEXT.build_vocab(cola_train_data, min_freq=2)


cola_train_iterator, cola_valid_iterator, cola_test_iterator = BucketIterator.splits(
    (cola_train_data, cola_valid_data, cola_test_data), 
    batch_size=32, 
    device=None,
    sort=False,
)


sat_train_data, sat_valid_data, sat_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="sat_train.tsv",
    validation="sat_valid.tsv",
    test="sat_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1
)

sat_train_iterator, sat_valid_iterator, sat_test_iterator = BucketIterator.splits(
    (sat_train_data, sat_valid_data, sat_test_data), 
    batch_size=8, 
    device=None,
    sort=False,
)

In [7]:
# LSTM Pooling Classifier
class LSTMPoolingClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super(LSTMPoolingClassifier, self).__init__()
        self.embed_layer = nn.Embedding(
            num_embeddings=num_embeddings, 
            embedding_dim=embedding_dim, 
            padding_idx=pad_idx
        )
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.lstm_layer = nn.LSTM(
            input_size = embedding_dim, 
            hidden_size = hidden_size, 
            num_layers = num_layers,
            bidirectional=True, 
            batch_first=True, 
            dropout=0.5
        )
        self.last_layer = nn.Sequential(
            nn.Linear(2 * hidden_size, 1),
            nn.Dropout(p=0.5),
            nn.Sigmoid()            
        )

    def forward(self, x):
        x = self.embed_layer(x) # Token으로 들어온 데이터를 Embedding하여 값으로 변환
        o, _ = self.lstm_layer(x) #변환된 값들을 lstm에 넣는다
        pool = nn.functional.max_pool1d(o.transpose(1, 2), x.shape[1]) # LSTM의 결과를 Max Pooling
        pool = pool.transpose(1, 2).squeeze() # Max Pooling한 결과를 Fully Connected Layer에 넣기위해 Shape를 맞춰준다.
        output = self.last_layer(pool) # Fully Connected Layer에 넣에서 결과를 반환
        return output.squeeze()

In [5]:
def train(model: nn.Module,
          iterator: Iterator,
          optimizer: torch.optim.Optimizer,
          criterion: nn.Module,
          device: str):
    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):
        optimizer.zero_grad()

        text = batch.text
        if text.shape[0] > 1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)

            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: Iterator,
             criterion: nn.Module,
             device: str):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def test(
    model: nn.Module,
    iterator: Iterator,
    device: str):

    with torch.no_grad():
        y_real = []
        y_pred = []
        model.eval()
        for batch in iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)

            output = model(text).flatten().cpu()

            y_real += [label]
            y_pred += [output]

        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc

def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [8]:
# 데이터 사전 학습(CoLA)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_pool_classifier = LSTMPoolingClassifier(
    num_embeddings=len(TEXT.vocab),
    embedding_dim=100,
    hidden_size=200,
    num_layers=4,
    pad_idx=PAD_IDX,
)

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_pool_classifier.to(device)

optimizer = torch.optim.Adam(lstm_pool_classifier.parameters())
bce_loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(
        lstm_pool_classifier, 
        cola_train_iterator, # CoLA를 활용하여 사전학습
        optimizer, 
        bce_loss_fn, 
        device
    )
    valid_loss = evaluate(
        lstm_pool_classifier, 
        cola_valid_iterator, 
        bce_loss_fn, 
        device
    )

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f}')
    print(f'\t Val. Loss: {valid_loss:.5f}')

Epoch: 01 | Time: 0m 53s
	Train Loss: 0.65626
	 Val. Loss: 0.63026
Epoch: 02 | Time: 0m 52s
	Train Loss: 0.64989
	 Val. Loss: 0.62818
Epoch: 03 | Time: 0m 49s
	Train Loss: 0.64406
	 Val. Loss: 0.61982
Epoch: 04 | Time: 0m 48s
	Train Loss: 0.63023
	 Val. Loss: 0.62201
Epoch: 05 | Time: 0m 48s
	Train Loss: 0.62568
	 Val. Loss: 0.61875
Epoch: 06 | Time: 0m 48s
	Train Loss: 0.61743
	 Val. Loss: 0.62346
Epoch: 07 | Time: 0m 48s
	Train Loss: 0.60302
	 Val. Loss: 0.61982
Epoch: 08 | Time: 0m 48s
	Train Loss: 0.58095
	 Val. Loss: 0.62177
Epoch: 09 | Time: 0m 48s
	Train Loss: 0.56808
	 Val. Loss: 0.63411
Epoch: 10 | Time: 0m 48s
	Train Loss: 0.54747
	 Val. Loss: 0.62520
Epoch: 11 | Time: 0m 48s
	Train Loss: 0.53947
	 Val. Loss: 0.64505
Epoch: 12 | Time: 0m 49s
	Train Loss: 0.52861
	 Val. Loss: 0.65238
Epoch: 13 | Time: 0m 49s
	Train Loss: 0.51570
	 Val. Loss: 0.63976
Epoch: 14 | Time: 0m 49s
	Train Loss: 0.49453
	 Val. Loss: 0.66625
Epoch: 15 | Time: 0m 49s
	Train Loss: 0.48547
	 Val. Loss: 0.6

In [9]:
# 성능 비교를 위한 모델 저장
before_tuning_lstm_pool_classifier = deepcopy(lstm_pool_classifier)

In [10]:
# 수능 데이터를 이용한 추가학습(Fine-tuning)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20


for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(lstm_pool_classifier, sat_train_iterator, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(lstm_pool_classifier, sat_valid_iterator, bce_loss_fn, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f}')
    print(f'\t Val. Loss: {valid_loss:.5f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 0.57829
	 Val. Loss: 0.52859
Epoch: 02 | Time: 0m 2s
	Train Loss: 0.55525
	 Val. Loss: 0.54332
Epoch: 03 | Time: 0m 2s
	Train Loss: 0.49988
	 Val. Loss: 0.52549
Epoch: 04 | Time: 0m 2s
	Train Loss: 0.45118
	 Val. Loss: 0.53627
Epoch: 05 | Time: 0m 2s
	Train Loss: 0.47491
	 Val. Loss: 0.52213
Epoch: 06 | Time: 0m 2s
	Train Loss: 0.35130
	 Val. Loss: 0.54819
Epoch: 07 | Time: 0m 1s
	Train Loss: 0.41929
	 Val. Loss: 0.48988
Epoch: 08 | Time: 0m 2s
	Train Loss: 0.46890
	 Val. Loss: 0.49537
Epoch: 09 | Time: 0m 2s
	Train Loss: 0.43886
	 Val. Loss: 0.51117
Epoch: 10 | Time: 0m 2s
	Train Loss: 0.40650
	 Val. Loss: 0.46684
Epoch: 11 | Time: 0m 2s
	Train Loss: 0.48885
	 Val. Loss: 0.45990
Epoch: 12 | Time: 0m 2s
	Train Loss: 0.45613
	 Val. Loss: 0.42199
Epoch: 13 | Time: 0m 2s
	Train Loss: 0.43603
	 Val. Loss: 0.42211
Epoch: 14 | Time: 0m 2s
	Train Loss: 0.36152
	 Val. Loss: 0.45158
Epoch: 15 | Time: 0m 1s
	Train Loss: 0.46594
	 Val. Loss: 0.39647
Epoch: 16 

In [11]:
# 모델의 성능 비교
_ = before_tuning_lstm_pool_classifier.cpu()
_ = lstm_pool_classifier.cpu()

pool_sat_test_auroc = test(before_tuning_lstm_pool_classifier, sat_test_iterator, "cpu")
pool_tuned_test_auroc = test(lstm_pool_classifier, sat_test_iterator, "cpu")

print(f"Before fine-tuning SAT Dataset Test AUROC: {pool_sat_test_auroc:.5f}")
print(f"After fine-tuning SAT Dataset Test AUROC: {pool_tuned_test_auroc:.5f}")

Before fine-tuning SAT Dataset Test AUROC: 0.69231
After fine-tuning SAT Dataset Test AUROC: 0.65385


In [12]:
# 모델 저장
with open("advanced_before_tuning_model.dill", "wb") as f:
    model = {
        "TEXT": TEXT,
        "LABEL": LABEL,
        "classifier": before_tuning_lstm_pool_classifier
    }
    dill.dump(model, f)

with open("advanced_after_tuning_model.dill", "wb") as f:
    model = {
        "TEXT": TEXT,
        "LABEL": LABEL,
        "classifier": lstm_pool_classifier
    }
    dill.dump(model, f)